# 5. Dense and sparse matrices

In this notebook in addition to NumPy we'll also be using SciPy. SciPy is a library for (mathematical) functions that are often used in scientific research.
It consists of function for optimization, interpolation, integration, linear algebra, statistics and other topics.
SciPy also has functions to work with sparse matrices.
Make sure SciPy is installed on your system. When using the Jupyter server of the university SciPy is already installed.
Since we will work in this notebook with the sparse submodule of the SciPy package we have to import this module separately.

For more information about SciPy visit the following website:
http://docs.scipy.org/doc/scipy/reference/tutorial/index.html

In [1]:
import numpy
import scipy
import scipy.sparse

## Dense versus sparse matrices


Sparseness can refer to either the **content** of a matrix, or to the **representation** of a matrix in the program.

### Dense and sparse content

A dense matrix is a matrix with a lot of nonzero elements.
A sparse matrix is the opposite and contains of a lot of zeros. 

The level of sparsity or density is an indication of how sparse or dense the matrix is.
The sparsity/density can be computed by dividing the number of nonzero elements by the total number of elements.
When this number is low, then the matrix contains a lot of zeros and therefore the matrix is called sparse.
When this number is high, then the matrix contains no or only a few zeros and therefore the matrix is called dense.

### Sparse matrix representations

When a matrix contains only a few nonzero elements then a specialized sparse representation of the matrix can be used. A sparse representation is a represenation in which only the nonzero elements are stored, while the zeros are implicit. There are several different sparse representations.

In the case where the matrix is large and sparse it will be beneficial to use a sparse representation of the matrix.
The advantage of using a sparse matrix representation are:
- Memory: less memory is needed to store the matrix since the zero elements are not stored.
- Efficiency: using a sparse matrix in functions or loops can speed up the process in comparison with using a dense representation of the matrix since the zero elements are skipped in the sparse representation.

Examples of situations when a sparse matrix structure is used:

- In the case of counting words in textual documents. A document-term matrix is created, which is a matrix where for every document a vector is representing the frequencies of the words that occur in that document. A document-term matrix has in general a lot of nzero values and therefore a sparse matrix structure is optimal.
- All diagonal matrices only have values on the diagonal and the other values are zeros, so these matrices can be better represented as a sparse matrix or alternatively as a vector of only the diagonal components.
- Upper or lower triangular matrix. Only approximately half of the records (including the diagonal) is non-zero and the other half is zeros. In the upper triangular matrix the upper part of the matrix has non-zero values and in the lower triangular matrix the lower part of the matrix has non-zero values. 

## Different ways to store a sparse matrix

There are several different sparse matrix classes in SciPy:
- Dictionary of keys (dok_matrix)
- List of lists (lil_matrix)
- Coordinate list (coo_matrix)
- Diagonal storage (dia_matrix)
- Compressed sparse column (csc_matrix)
- Compressed sparse row (csr_matrix)
- Block sparse row matrix (bsr_matrix)


If you want to create a sparse matrix incrementally, it's best to use `dok` or `lil`.
You should then convert it to `csc` or `csr` for efficient matrix operations, such as matrix multiplication or inversion.

Below some different sparse matrix classes will be discussed separately.
For more information: http://docs.scipy.org/doc/scipy/reference/sparse.html

### Dictionary of keys (dok)

As the name already indicates the dictionary of keys is a dictionary format where every key in the dictionary represents the row and column indices of the element and the value in the dictionary represents the value of the element at that particular position. Elements with value 0 are not present in the dictionary, but they are assumed to be implicitly zero. 



In [2]:
# Create a dok matrix incrementally

S = scipy.sparse.dok_matrix((5, 5), dtype=numpy.int32)
for i in range(5):
    for j in range(5):
        S[i,j] = i*j
print(S)

  (1, 2)	2
  (3, 2)	6
  (1, 3)	3
  (3, 3)	9
  (4, 1)	4
  (3, 1)	3
  (4, 4)	16
  (2, 1)	2
  (2, 4)	8
  (2, 3)	6
  (1, 4)	4
  (4, 3)	12
  (2, 2)	4
  (4, 2)	8
  (3, 4)	12
  (1, 1)	1


### Exercise 5.1

In this exercise we will learn how to build a document-term matrix.
We'll build two functions.

- Create function `word_index`  which takes a list of words, and returns a dictionary mapping each unique word to an index. For example:
```python
word_index("a rose is a rose".split())
```
```
{'A': 0, 'a': 3, 'is': 2, 'rose': 1}
```
- Create function `word_count` which takes a list of texts (where text is a list of words), and returns two values:
   - the dictionary mapping words to indices
   - the `dok_matrix` which counts how many times each word occurs in each text. The matrix should be $M\times N$, where $M$ is the number of texts and  $N$ is the total number word types (i.e. total number of unique words). In your implementation you will use the function `word_index` to map words to indices.

Example:

```python
text = ['All human beings are born free and equal in dignity and rights'.split(),
       'They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood'.split()]
vocab, M = word_count(text)
print(M[0, vocab['and']])
print(M[1, vocab['reason']])
```       
Output:
```
2.0
1.0
```

### Compressed sparse row (csr)

CSR represents the matrix by three arrays, that contain the non-zero values, the extents of rows and the column indices. 

scipy.sparse.csr_matrix(D, shape=(nrows, ncols), dtype=)

Advantages: efficient arithmetic operations, efficient row slicing and row operations, and fast matrix vector products. 

Disadvantages: slow column slicing operations (use CSC for faster column slicing), and changes to the sparsity structure are expensive (use LIL or DOK instead). 


In [11]:
D = numpy.round(numpy.random.random((3,3)))
S = scipy.sparse.csr_matrix(D, shape=(3,3), dtype=numpy.int32)
print(S)

  (0, 0)	1
  (1, 0)	1
  (1, 2)	1
  (2, 0)	1


### Compressed sparse column (csc)

Similar to Compressed sparse row (CSR) matrices, but then orientated to the columns. 

scipy.sparse.csc_matrix(D, shape=(nrows, ncols), dtype=)

Advantages: efficient arithmetic operations, efficient column slicing, and fast matrix vector products (but CSR may be faster).

Disadvantages: slow row slicing operations (use CSR instead), and changes tot he sparsity structure are expensive (use LIL or DOK). 


In [12]:
D = numpy.round(numpy.random.random((3,3)))
S = scipy.sparse.csc_matrix(D, shape=(3,3), dtype=numpy.int32)
print(S)

  (1, 0)	1
  (2, 0)	1
  (0, 1)	1
  (1, 1)	1
  (2, 1)	1
  (2, 2)	1


## Functions to create a sparse matrix

When in a built-in function the format is needed, then the three letters abbreviations of the sparse representations are used as string, which are `dok, lil, coo, csr, csc, dia, and bsr`. 

There are several built-in functions to create a special matrix in a sparse representation. Some examples:
- Create the m by n identity matrix with the ones on the diagonal (k=0 is the main diagonal):  scipy.sparse.eye(m, n, k, dtype=, format= )
- Create a m by n matrix with random elements (floats between 0 and 1) and the density of the matrix can be given: scipy.sparse.rand(m, n, density=, format=, dtype= )

In [13]:
# sparse identity matrix
E = scipy.sparse.eye(5, 5, 0, dtype=numpy.int32, format="dok" )
print(E)

  (3, 3)	1
  (0, 0)	1
  (1, 1)	1
  (4, 4)	1
  (2, 2)	1


In [14]:
# sparse random matrix
R = scipy.sparse.rand(5,5,density=0.2, format="coo", dtype=numpy.float32)
print(R)

  (1, 2)	0.420558
  (2, 0)	0.12975
  (3, 1)	0.505013
  (2, 1)	0.0733214
  (0, 2)	0.550031


## Check if the matrix is sparse

When you want to check whether or not the matrix is a sparse format then the following function can be helpful:
- `scipy.sparse.issparse(x)`
- `scipy.sparse.isspmatrix(x)`
- `scipy.sparse.isspmatrix_dok(x) / scipy.sparse.isspmatrix_lil(x) / scipy.sparse.isspmatrix_coo(x)`: the function is the same for all different formats and everytime the three letters abbreviations are used. 

All these functions return True in the case where the matrix is sparse and of the particular type and False otherwise.

In [15]:
R = scipy.sparse.rand(5,5,density=0.2, format="coo", dtype=numpy.float32)
print("Is sparse?", scipy.sparse.issparse(R))
print("Is Dictionary of Keys format?", scipy.sparse.isspmatrix_dok(R))
print("Is Coordinate lists format?", scipy.sparse.isspmatrix_coo(R) )

Is sparse? True
Is Dictionary of Keys format? False
Is Coordinate lists format? True


## Using the sparse representations

You can use the matrices with sparse represenations in normal mathematical transformations, such as addition, substraction, division, multiplication, and matrix power.

If you want to elementwise-multiply two sparse matrices, you can use the x.multiply(y) function.

http://docs.scipy.org/doc/scipy/reference/sparse.html

In [16]:
# mathematical transformations with sparse matrices
x = scipy.sparse.rand(4,4,density=0.5, format="csr", dtype=numpy.float32)
y = scipy.sparse.rand(4,4,density=0.5, format="csr", dtype=numpy.float32)
print("x=")
print(x)
print("y=")
print(y)
print("x+y=")
print(x+y)
print("x*y=")
print(x*y)
print("elementwise multiplication:", x.multiply(y))

x=
  (0, 1)	0.618135
  (1, 0)	0.591079
  (1, 2)	0.348162
  (2, 0)	0.115945
  (2, 1)	0.0484889
  (2, 2)	0.81804
  (3, 0)	0.240708
  (3, 3)	0.14134
y=
  (0, 1)	0.397419
  (0, 3)	0.738682
  (1, 0)	0.874371
  (1, 3)	0.983945
  (2, 0)	0.855514
  (2, 2)	0.465812
  (3, 2)	0.672625
  (3, 3)	0.0525603
x+y=
  (0, 1)	1.01555
  (0, 3)	0.738682
  (1, 0)	1.46545
  (1, 2)	0.348162
  (1, 3)	0.983945
  (2, 0)	0.97146
  (2, 1)	0.0484889
  (2, 2)	1.28385
  (3, 0)	0.240708
  (3, 2)	0.672625
  (3, 3)	0.1939
x*y=
  (0, 3)	0.608211
  (0, 0)	0.54048
  (1, 2)	0.162178
  (1, 0)	0.297858
  (1, 3)	0.43662
  (1, 1)	0.234906
  (2, 2)	0.381053
  (2, 0)	0.742242
  (2, 3)	0.133357
  (2, 1)	0.0460789
  (3, 2)	0.0950689
  (3, 3)	0.185236
  (3, 1)	0.0956619
elementwise multiplication:   (0, 1)	0.245659
  (1, 0)	0.516822
  (2, 0)	0.0991929
  (2, 2)	0.381053
  (3, 3)	0.00742888


## Often used functions applied to sparse matrices

- Calculating the mean of the matrix. For example, when the matrix is a dictionary of keys sparse matrix then the following function can be used to calculate the mean: `scipy.sparse.dok_matrix.mean(axis=0)` and this function does not destroy the sparse matrix structure. The right way to use this function is to first create a matrix x which is a sparse matrix of the `dok` type and then using `x.mean()` to calculate the mean. The parameter is the axis and this is helful when you want to calculate the mean for every row or column. When calculating the mean for every column separately one has to set `axis=0` and when one wants to calculate the mean for every row separately then one has to set `axis=1`. Other types of sparse matrices have similar functions, such as `scipy.sparse.lil_matrix.mean()`, and `scipy.sparse.coo_matrix.mean()`.


- Calculating the minimum or maximum of the matrix without destroying the sparse structure. For these function the same holds as for the mean function, so one has to first create the sparse matrix of a certain format and then apply the function to this matrix. For example, create a sparse matrix x in the dictionary of keys format and then `x.minimum()` will give the minimum over the matrix. 

Other functions that can be applied to sparse matrices (where x is a sparse matrix):
- `x.transpose()` or `x.T`: return the transpose of the matrix as a sparse matrix.
- `x.toarray()`: convert sparse matrix into a regular numpy array
- `x.todense()`:  convert sparse matrix to densely represented matrix (not the same as a numpy array!)
- `x.sum(axis=)`: calculate the sum over the matrix or the sum over one axis.
- `x.nonzero()`: return the indices of the non-zero elements.
- `x.reshape(shape)`: reshape the array to another shape.
- `x.multiply(y)`: pointwise multiplication of x with another matrix y (be careful with the shape).


In [17]:
# create a random 10 by 5 sparse matrix with density 0.4 in the dictionary of keys format 
x = scipy.sparse.rand(10,5,density=0.4, format="dok", dtype=numpy.float32)

# calculate the mean without destroying the sparse structure
print("Mean over the total matrix: ", x.mean())
print("Mean for every column: ", x.mean(axis=0))

# calculate the sum over the matrix
print("Total sum of the matrix:", x.sum())

Mean over the total matrix:  0.181695098877
Mean for every column:  [[ 0.18655983  0.26865026  0.09759878  0.14443907  0.21122761]]
Total sum of the matrix: 9.08475


In [18]:
# create sparse matrix
x = scipy.sparse.rand(10,5,density=0.4, format="dok", dtype=numpy.float32)

# transpose
print("Transpose: ", x.transpose())

# multiply pointwise
y = scipy.sparse.rand(10, 5,density=0.2, format="dok", dtype=numpy.float32)
print("Multiply pointwise: ", x.multiply(y))

# transform to dense matrix
print("Dense: ", x.todense())

Transpose:    (1, 2)	0.324699
  (4, 7)	0.0911715
  (1, 3)	0.349818
  (4, 9)	0.259366
  (3, 0)	0.253152
  (3, 4)	0.826556
  (3, 1)	0.935196
  (3, 2)	0.995091
  (2, 2)	0.292325
  (3, 8)	0.699986
  (0, 6)	0.401445
  (2, 6)	0.0814487
  (2, 3)	0.403576
  (1, 7)	0.463117
  (0, 9)	0.945913
  (1, 6)	0.702696
  (2, 5)	0.993092
  (4, 1)	0.320977
  (1, 1)	0.61328
  (4, 0)	0.122833
Multiply pointwise:    (0, 3)	0.167365
  (1, 1)	0.129415
  (1, 3)	0.107245
  (7, 4)	0.0720837
Dense:  [[ 0.          0.          0.          0.25315207  0.1228329 ]
 [ 0.          0.61328     0.          0.93519646  0.32097661]
 [ 0.          0.32469872  0.29232466  0.99509132  0.        ]
 [ 0.          0.34981787  0.40357637  0.          0.        ]
 [ 0.          0.          0.          0.82655579  0.        ]
 [ 0.          0.          0.99309206  0.          0.        ]
 [ 0.40144524  0.70269644  0.08144868  0.          0.        ]
 [ 0.          0.46311659  0.          0.          0.09117148]
 [ 0.          0.    

## Important warnings

### Mean subtraction
When subtracting the mean from all values of a sparse matrix one has to be very careful because with this operation the sparseness of the matrix is immediately destroyed. The reason is that from all zeros values the mean in subtracted so in general this will lead to a matrix with almost no zero values and all the old zero values will now have the value -mean (and of course the only exception is when the mean is equal to zero). 
The same warning hold for normalization or standardization of the matrix.

In [12]:
# the result of mean substraction of a sparse matrix is a dense matrix
x = scipy.sparse.rand(10,5,density=0.4, format="dok", dtype=numpy.float32)
print("mean: ", x.mean())
print(x-x.mean())

mean:  0.177649


NotImplementedError: adding a nonzero scalar to a sparse matrix is not supported

As you see, SciPy makes it easy to avoid this operation, since it throws an error when you attempt it :-) 

### Dot product between sparse matrices
When using sparse matrices you should not use the code: `numpy.dot(x,y)` to get the dot product between a matrix x and a vector y. Use `x.dot(y)` instead.
Sparse matrices are not supported in `numpy.dot(x,y)` and therefore this can lead to error messages. 
Reference: http://docs.scipy.org/doc/scipy/reference/sparse.html#matrix-vector-product


In [8]:
# dot product 
x = scipy.sparse.dok_matrix([[1, 2, 0], [0, 0, 3], [4, 0, 5]])
y = numpy.array([5, 6, 0])
print(x.dot(y))
print()
# DO NOT DO THIS
print(numpy.dot(x, y))

[17  0 20]

[ <3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Dictionary Of Keys format>
 <3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Dictionary Of Keys format>
 <3x3 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Dictionary Of Keys format>]


/usr/local/lib/python3.4/dist-packages/scipy/sparse/compressed.py:294: SparseEfficiencyWarning: Comparing sparse matrices using >= and <= is inefficient, using <, >, or !=, instead.
  "using <, >, or !=, instead.", SparseEfficiencyWarning)


### Missing operations

Certain operations are not available for sparse matrices: either because they are not compatible with sparsity, or they simply haven't been implemented. So you may find that you occasionaly need to convert a sparse matrix to a numpy array (using `.toarray()`) to perform an operation. Be careful when doing this with very large matrices: they will most likely not fit in memory, and crash your python session.

Some commonly used missing functions include:

- `.argmax`
- `.argmin`
- `.argsort`

### Exercise 5.2

Given a word index dictionary and a document-term matrix returned by `word_count`, find the most frequent word for each row.

### Exercise 5.3

Function `f` takes a document-term matrix, and returns another matrix. Analyze the code in `f` and try to figure out what this function is doing and how to interpret the result it returns.
```python
def f(D):
    M = D.minimum(1.0)
    return M.dot(M.T)
```

### Exercise 5.4

- Create a list of texts by reading the first 1000 lines from [coco_val.txt](coco_val.txt). Create matrix A with the document-term matrix from these 1000 sentences. Create another matrix B by converting the matrix A from `dok` format to `csr` format. Use the ipython command `%timeit` to compare whether it's faster to apply function `f` (see above) to matrix A or B.
- Create matrix C by converting A to a regular numpy array. Write function `g` which is like `f`, but works on regular numpy array arrays. Use `%timeit` to see how fast g is when applied to matrix C.

### Exercise 5.5

There are a number of ways of measuring similarity between documents. For this exercise we'll try to use the number of unique words the documents have in common (word overlap).


1. Define function `similarity` which takes a document-term matrix, and returns the matrix where the value in the row $i$ and column $j$ is the word overlap between the $i$th and $j$th document.
3. Calculate the overlap similarity matrix for sentences in [coco_val.txt](coco_val.txt). For the first 20 sentences, display the 3 most similar sentences according to this matrix.
4. Word overlap is a very simplistic similarity measure. Suggest, and possibly implement, a better way of measuring similarity between documents based on a document-term matrix.